In [ ]:
import pathlib

import astropy.coordinates as coord
import astropy.table as at
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

# gala
import gala.coordinates as gc
import gala.dynamics as gd
import gala.potential as gp
from gala.units import galactic

from pyia import GaiaData

# from dustmaps.leike2020 import Leike2020Query

In [ ]:
vcirc = 229 * u.km/u.s
galcen_frame = coord.Galactocentric()
galcen_frame

In [ ]:
t1 = at.Table.read(pathlib.Path('~/data/GaiaEDR3/edr3-rv-good-plx-result.fits'))
t2 = at.Table.read(pathlib.Path('~/data/GaiaEDR3/edr3-rv-good-plx-2mass-xm.fits.gz'))
t = at.join(t1, t2, keys='source_id')

In [ ]:
g = GaiaData(t)
c = g.get_skycoord()

In [ ]:
iso = at.Table.read('/Users/apricewhelan/data/Isochrones/mist/mist_v1.2_vvcrit0.0.fits')

In [ ]:
MG = g.get_G0() - g.distmod
BPRP = g.get_BP0() - g.get_RP0()

# MG = g.phot_g_mean_mag - g.distmod
# BPRP = g.phot_bp_mean_mag - g.phot_rp_mean_mag
# GRP = g.phot_g_mean_mag - g.phot_rp_mean_mag

In [ ]:
i = iso[(iso['log10_isochrone_age_yr'] == 8.8) & 
        (iso['[Fe/H]_init'] == 0.5) & 
        (iso['phase'] < 1)]
len(i)

In [ ]:
mask = (
    (g.ruwe < 1.2) &
    (np.abs(g.b) > 5*u.deg) &
    (BPRP != 0.)
)


fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.hist2d(
    BPRP.value[mask], MG.value[mask], 
    bins=(np.linspace(-1, 3, 256),
          np.linspace(-5, 10, 256)),
    norm=mpl.colors.LogNorm()
)

ax.plot(i['Gaia_BP_EDR3'] - i['Gaia_RP_EDR3'], 
        i['Gaia_G_EDR3'], 
        marker='')

ax.set_xlim(-1., 3)
ax.set_ylim(10, -6)